<a href="https://colab.research.google.com/github/ImranNust/AppliedDataScience-CapstonProject/blob/main/Chapter4/Module1_WorkingWithImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> <center> <b> <u> Important Points </u> </b> </center> </h1>



* Scalars representing values at individual pixels are often encoded using 8-bit integers. 
* In medical, scientific, and industrial applications, it is not unusual to find higher numerical precision, such as 12-bit or 16-bit. This allows a wider range or increased sensitivity in cases where the pixel encodes information about a physical property, like bone density, temperature, or depth.

---

<h2> <center> <b> <u> Loading an Image </u> </b> </center> </h2>

In [10]:
import imageio
try:
  print("[INFO] Images folder already exists; therefore, reading the image directly")
  img_file = 'Images/bobby.jpg'
  img_arr = imageio.imread(img_file)
except:
  print("[INFO] Images folder does not exist; therefore, dowonloading it.. cheers...")
  !git clone https://github.com/ImranNust/DeepLearningWithPyTorch
  !mv DeepLearningWithPyTorch/Images .
  !rm -rf DeepLearningWithPyTorch
  img_file = 'Images/bobby.jpg'
  img_arr = imageio.imread(img_file)


print('Image Size: {}'.format(img_arr.shape))

[INFO] Images folder already exists; therefore, reading the image directly
Image Size: (720, 1280, 3)


---
<h2> <center> <b> <u> Changing the Layout </u> </b> </center> </h2>

img is a NumPy array-like object with three dimensions: $H\times W\times C$. 
PyTorch modules dealing with image data require tensors to be laid out as $C\times H\times W$. We will, therefore, use the `permute' function to make the dimensions PyTorch Compatible.

---

In [11]:
import torch

# coverting numpy to pytorch tensors
img = torch.from_numpy(img_arr) 

# permuting to bring the channels to the first position
out = img.permute(2, 0, 1)

print('Now the size is {}'.format(out.shape))




Now the size is torch.Size([3, 720, 1280])


---

[IMPORTANT]: note that this operation does not make a copy of the tensor data. Instead, `out` uses the same underlying storage as `img` and only plays with The size and stride information at the tensor level. This is convenient because the operation is very cheap; but just as a heads-up: **changing a pixel in img will lead to a change in out.**

---

---

Let's build a tensor, using `stack`, where we can pre-allocate a tensor of appropriate size and fill it with images loaded from a directory.

---

In [12]:
import os
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype = torch.uint8)

"""
This indicates that our batch will consist of three RGB images 256 pixels in 
height and 256 pixels in width.
"""

# We can now load all PNG images from an input directory and store them in the 
# tensor
data_dir = 'Images/image-cats/'
filenames = [name for name in os.listdir(data_dir)
if os.path.splitext(name)[-1] == '.png']

for i, filename  in enumerate(filenames):
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  img_t = img_t[:3]
  batch[i] = img_t

---
<h2> <center> <b> <u> Normalizing the Data </u> </b> </center> </h2>

* Neural networks exhibit the best training performance when the input data ranges roughly from 0 to 1, or from -1 to 1. 

* So a typical thing we’ll want to do is cast a tensor to floating-point and normalize the values of the pixels. Casting to floating-point is easy, but normalization is trickier, as it depends on what range of the input we decide should lie between 0 and 1 (or -1 and 1). One possibility is to just divide the values of the pixels by 255 (the maximum representable number in 8-bit unsigned):

```
batch = batch.float()
batch /= 255.0
```

Another possibility is to compute the mean and standard deviation of the input data and scale it so that the output has zero mean and unit standard deviation across each channel:


---

In [13]:
batch = batch.float()
batch /= 255.0
n_channels = batch.shape[1]
for c in range(n_channels):
  mean = torch.mean(batch[:, c])
  std = torch.std(batch[:, c])
  batch[:, c] = (batch[:, c] - mean) / std

---

[IMPORTANT]: NOTE Here, we normalize just a single batch of images because we do not know yet how to operate on an entire dataset. In working with images, it is good practice to compute the mean and standard deviation on all the training data in advance and then subtract and divide by these fixed, precomputed quantities.

---